In [1]:
!pip install PyGithub

In [1]:
from github import Github
import os
import base64
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ipywidgets import interact
import datetime as dt
sns.set()

In [2]:
with open("github.txt") as myfile:
    firstNlines=myfile.readlines()[0:2]
myfile.close()

In [3]:
g = Github(firstNlines[0].strip(), firstNlines[1])

In [4]:
repo=g.get_repo('CSSEGISandData/COVID-19')

In [5]:
contents = repo.get_contents("")

In [6]:
contents[3]

ContentFile(path="csse_covid_19_data")

In [7]:
def get_sha_for_tag(repository, tag):
    """
    Returns a commit PyGithub object for the specified repository and tag.
    """
    branches = repository.get_branches()
    matched_branches = [match for match in branches if match.name == tag]
    if matched_branches:
        return matched_branches[0].commit.sha

    tags = repository.get_tags()
    matched_tags = [match for match in tags if match.name == tag]
    if not matched_tags:
        raise ValueError('No Tag or Branch exists with that name')
    return matched_tags[0].commit.sha

In [8]:
def download_directory(repository, sha, server_path):
    """
    Download all contents at server_path with commit tag sha in
    the repository.
    """
    contents = repository.get_contents(server_path, ref=sha)

    for content in contents:
        print("Processing %s" % content.path)
        if content.type == 'dir':
            download_directory(repository, sha, content.path)
        else:
            try:
                path = content.path
                file_content = repository.get_contents(path, ref=sha)
                file_data = base64.b64decode(file_content.content).decode('ascii')
                file_out = open(content.name, "w")
                file_out.write(file_data)
                file_out.close()
            except:
                pass

In [9]:
sha = get_sha_for_tag(repo, 'master')

In [10]:
download_directory(repo, sha, 'csse_covid_19_data/csse_covid_19_daily_reports')

Processing csse_covid_19_data/csse_covid_19_daily_reports/.gitignore
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-22-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-23-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-24-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-25-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-26-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-27-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-28-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-29-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-30-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-31-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/02-01-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/02-02-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_repor

In [11]:
df = pd.read_csv(r'C:\Users\brian\projects\COVID-19\csse_daily_reports\02-01-2020.csv')

In [12]:
from os import listdir
from os.path import isfile, join
mypath = r'C:\Users\brian\projects\COVID-19\csse_daily_reports\\'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [13]:
def is_non_zero_file(fpath):
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

In [14]:
nonzero_files = []
for file in onlyfiles:
    if is_non_zero_file(mypath + file) and file[-3:]=='csv':
        nonzero_files.append(mypath + file)

In [15]:
nonzero_files

['C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-01-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-02-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-03-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-04-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-05-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-06-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-07-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-08-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-09-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-10-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-11-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-12-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_r

In [16]:
df_list = []
for file in nonzero_files:
    df_1 = pd.read_csv(file)
    df_1['filename'] = file[-15:]
    df_list.append(df_1)  

In [17]:
df = pd.concat(df_list, ignore_index=True)

C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4020 entries, 0 to 4019
Data columns (total 9 columns):
Confirmed         4020 non-null int64
Country/Region    4020 non-null object
Deaths            4020 non-null int64
Last Update       4020 non-null object
Latitude          1695 non-null float64
Longitude         1695 non-null float64
Province/State    2522 non-null object
Recovered         4020 non-null int64
filename          4020 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 282.8+ KB


In [19]:
df['Date'] = pd.to_datetime(df['Last Update'])

In [20]:
df.drop(columns=['Latitude', 'Longitude'], inplace=True)

In [21]:
df['Diamond Princess'] = df['Province/State'].str.contains('Diamond Princess')

In [22]:
df.loc[df['Province/State']=='From Diamond Princess']

,Confirmed,Country/Region,Deaths,Last Update,Province/State,Recovered,filename,Date,Diamond Princess
1499,4,Australia,0,2020-02-21T23:03:13,From Diamond Princess,0,\02-21-2020.csv,2020-02-21 23:03:13,True
1518,1,Israel,0,2020-02-21T15:33:03,From Diamond Princess,0,\02-21-2020.csv,2020-02-21 15:33:03,True
1580,7,Australia,0,2020-02-22T17:03:05,From Diamond Princess,0,\02-22-2020.csv,2020-02-22 17:03:05,True
1602,1,Israel,0,2020-02-22T20:53:02,From Diamond Princess,0,\02-22-2020.csv,2020-02-22 20:53:02,True
1664,7,Australia,0,2020-02-22T17:03:05,From Diamond Princess,0,\02-23-2020.csv,2020-02-22 17:03:05,True
1686,1,Israel,0,2020-02-22T20:53:02,From Diamond Princess,0,\02-23-2020.csv,2020-02-22 20:53:02,True
1749,7,Australia,0,2020-02-22T17:03:05,From Diamond Princess,0,\02-24-2020.csv,2020-02-22 17:03:05,True
1773,1,Israel,0,2020-02-22T20:53:02,From Diamond Princess,0,\02-24-2020.csv,2020-02-22 20:53:02,True
1841,7,Australia,0,2020-02-22T17:03:05,From Diamond Princess,0,\02-25-2020.csv,2020-02-22 17:03:05,True
1867,1,Israel,0,2020-02-22T20:53:02,From Diamond Princess,0,\02-25-2020.csv,2020-02-22 20:53:02,True


In [23]:
df.drop(columns=['filename'], inplace=True)

In [24]:
df = df.drop_duplicates()

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2208 entries, 0 to 3999
Data columns (total 8 columns):
Confirmed           2208 non-null int64
Country/Region      2208 non-null object
Deaths              2208 non-null int64
Last Update         2208 non-null object
Province/State      1492 non-null object
Recovered           2208 non-null int64
Date                2208 non-null datetime64[ns]
Diamond Princess    1492 non-null object
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 155.2+ KB


In [26]:
df['Province/State'] = df['Province/State'].str.replace(' \(From Diamond Princess\)', '').replace('None', 'From Diamond Princess')

In [27]:
df.head()

,Confirmed,Country/Region,Deaths,Last Update,Province/State,Recovered,Date,Diamond Princess
0,7153,Mainland China,249,2/1/2020 11:53,Hubei,168,2020-02-01 11:53:00,False
1,599,Mainland China,0,2/1/2020 10:53,Zhejiang,21,2020-02-01 10:53:00,False
2,535,Mainland China,0,2/1/2020 14:23,Guangdong,14,2020-02-01 14:23:00,False
3,422,Mainland China,2,2/1/2020 1:52,Henan,3,2020-02-01 01:52:00,False
4,389,Mainland China,0,2/1/2020 11:03,Hunan,8,2020-02-01 11:03:00,False


In [28]:
df['DateTime'] = df['Date']

In [29]:
df['Date'] = df['DateTime'].apply(lambda x: x.date())

In [55]:
df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))

0      2020-02-01
1      2020-02-01
2      2020-02-01
3      2020-02-01
4      2020-02-01
          ...    
3978   2020-03-09
3980   2020-03-09
3993   2020-03-09
3997   2020-03-09
3999   2020-03-09
Name: Date, Length: 2208, dtype: datetime64[ns]

In [30]:
df.head()

,Confirmed,Country/Region,Deaths,Last Update,Province/State,Recovered,Date,Diamond Princess,DateTime
0,7153,Mainland China,249,2/1/2020 11:53,Hubei,168,2020-02-01,False,2020-02-01 11:53:00
1,599,Mainland China,0,2/1/2020 10:53,Zhejiang,21,2020-02-01,False,2020-02-01 10:53:00
2,535,Mainland China,0,2/1/2020 14:23,Guangdong,14,2020-02-01,False,2020-02-01 14:23:00
3,422,Mainland China,2,2/1/2020 1:52,Henan,3,2020-02-01,False,2020-02-01 01:52:00
4,389,Mainland China,0,2/1/2020 11:03,Hunan,8,2020-02-01,False,2020-02-01 11:03:00


In [31]:
df['Active'] = df['Confirmed']-df['Deaths']-df['Recovered']

In [77]:
df.loc[df['Country/Region']=='South Africa']

,Confirmed,Country/Region,Deaths,Last Update,Province/State,Recovered,Date,Diamond Princess,DateTime,Active
3048,1,South Africa,0,2020-03-05T13:53:03,NaN,0,2020-03-05,NaN,2020-03-05 13:53:03,1
3236,1,South Africa,0,2020-03-06T15:43:02,NaN,0,2020-03-06,NaN,2020-03-06 15:43:02,1
3641,3,South Africa,0,2020-03-08T10:23:04,NaN,0,2020-03-08,NaN,2020-03-08 10:23:04,3


In [75]:
@interact(Country = df['Country/Region'].sort_values().unique(), Confirmed=True, Active=True, Recovered=True, Deaths=True)
def viz(Country, Confirmed, Active, Recovered, Deaths):
    fig = px.line(title='Cases Over Time')
    if Confirmed:
        country_df = pd.DataFrame(df.loc[df['Country/Region']==Country].groupby('Date')['Confirmed'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Confirmed'], mode='lines', name='Confirmed Cases')
    if Active:
        country_df = pd.DataFrame(df.loc[df['Country/Region']==Country].groupby('Date')['Active'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Active'], mode='lines', name='Active Cases')
    if Recovered:
        country_df = pd.DataFrame(df.loc[df['Country/Region']==Country].groupby('Date')['Recovered'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Recovered'], mode='lines', name='Recovered Cases')
    if Deaths:
        country_df = pd.DataFrame(df.loc[df['Country/Region']==Country].groupby('Date')['Deaths'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Deaths'], mode='lines', name='Deceased Cases')
    fig.update_xaxes(title='Date', range=[dt.date(2020, 2, 1), dt.datetime.now()])
    fig.show()

interactive(children=(Dropdown(description='Country', options=(' Azerbaijan', 'Afghanistan', 'Albania', 'Alger…

In [79]:
@interact(Country = df['Country/Region'].sort_values().unique())
def viz(Country):
    country_df = df.loc[df['Country/Region']==Country].groupby('Date')['Confirmed', 'Active', 'Recovered', 'Deaths'].agg('sum')
    fig = px.line(title='Cases Over Time')
    fig.add_scatter(x = country_df.index, y = country_df['Confirmed', 'Active', 'Recovered', 'Deaths'], mode='lines', name='Confirmed Cases')
    fig.show()

interactive(children=(Dropdown(description='Country', options=(' Azerbaijan', 'Afghanistan', 'Albania', 'Alger…

In [80]:
df.loc[df['Country/Region']==Country].groupby('Date')['Confirmed', 'Active', 'Recovered', 'Deaths'].agg('sum')

NameError: name 'Country' is not defined